In [23]:
## Stoichiometric Reactor, with recycle (and purge), material balance only
## Yicheng Hu, Victor M. Zavala, 2017
using JuMP
using Ipopt

# define algebraic model
m = Model(solver = IpoptSolver(tol = 0.01, max_iter = 15000));

# define sets 
K = ["NH3","H2","N2","CH4","AR"];         # set of components
F = ["ℓ1","ℓ2","ℓ3","ℓ4","ℓ5","ℓ6","ℓ7"]; # set streams

# define data
μf = [0; 30; 10; 0.001; 0.001]  # input flows [mol/s]
μf = Dict(zip(K,μf));           # assign to set          

# define variables
@variable(m,      μ[K,F]>=0)  # molar flows
@variable(m,    μ_tot[F]>=0)  # molar flows
@variable(m, 0 <= y[K,F] <= 1); # molar fractions 

# compute total flows and mol fractions
  @constraint(m, [ℓ in F],         μ_tot[ℓ] == sum(μ[k,ℓ] for k in K));
@NLconstraint(m, [k in K, ℓ in F],   y[k,ℓ] == μ[k,ℓ]/μ_tot[ℓ]);
            
# Block1: Mixer
@constraint(m, [k in K], μ[k,"ℓ1"] == μf[k]); 
@constraint(m, [k in K], μ[k,"ℓ2"] == μ[k,"ℓ1"] + μ[k,"ℓ7"]);

# Block2: Reactor
       γ = [2/3; -1; -1/3; 0; 0]; # stochiometric coefficients
       γ = Dict(zip(K,γ));        # assign to set          
    klim = "H2";                  # limiting component
@variable(m, 0<=η<=0.7);          # extent of reaction 
            
@constraint(m, [k in K], μ[k,"ℓ3"] == μ[k,"ℓ2"] + η * (γ[k]/abs(γ[klim])) * μ[klim,"ℓ2"]);

# Block3: Flash separator
   ξf = [0.1;0.99;0.99;0.95;0.95];  # split fraction for each component
   ξf = Dict(zip(K,ξf));            # assign to set

@constraint(m, [k in K], μ[k,"ℓ3"] == μ[k,"ℓ4"] + μ[k,"ℓ5"]);
@constraint(m, [k in K], μ[k,"ℓ4"] == ξf[k]*μ[k,"ℓ3"]);

# Block4: Splitter
@variable(m, 0<=ξs<=1);             # split fraction 
@constraint(m, [k in K], μ[k,"ℓ6"] == μ[k,"ℓ4"] * ξs);
@constraint(m, [k in K], μ[k,"ℓ7"] == μ[k,"ℓ4"]*(1-ξs));
            
# objective function
@objective(m, Max, μ["NH3","ℓ5"])
            
# purity constraint
@constraint(m, y["NH3","ℓ5"] >= 0.98)
            
# recycle flow constraint
@constraint(m, μ_tot["ℓ7"] <= 1)            

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      247
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:      120

Total number of variables............................:       79
                     variables with only lower bounds:       42
                variables with lower and upper bounds:       37
                     variables with only upper bounds:        0
Total number of equality constraints.................:       77
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

In [24]:
println(getvalue(η))
println(getvalue(ξs))
println(getvalue(μ["NH3","ℓ5"]))
println(getvalue(y["NH3","ℓ5"]))
println(getvalue(μ_tot["ℓ7"]))

0.6999995223679188
0.9264150407737931
12.977005135699192
0.9906266556912299
0.999974513869612


In [25]:
println(getvalue(μ_tot))

μ_tot: 1 dimensions:
[ℓ1] = 40.001999999999995
[ℓ2] = 41.00197451386961
[ℓ3] = 26.689181294864667
[ℓ4] = 13.589387279384209
[ℓ5] = 13.099794015480462
[ℓ6] = 12.589412765514593
[ℓ7] = 0.999974513869612
